## Imports

In [ ]:
import ga_polynomial_simple as ga
import plotly.express as px
import logging
import numpy as np
import pandas as pd

In [ ]:
logging.getLogger().setLevel(logging.WARN)

## Visualising Target Polynomial

In [ ]:
figTargetPolynomial = px.line(x=ga.X_VALUES, y=ga.Y_TARGETS, title='Target Polynomial')
figTargetPolynomial.show()

## Producing GA Predictions

#### Setup Parameters

In [ ]:
count = 100
individualLength = 6
individualMin = -50
individualMax = 50

retain = 0.15
mutate = 0.1
random_select = 0.1

#### Running with selecion by roulette wheel

Specify GA training configuration

In [ ]:
maxGen = 400

Create first population and grade

population = ga.createPopulation(count, individualMin, individualMax, individualLength)
fitness_history = [ga.getAvgPopulationFitness(population)]

Show some data for sanity check

print(" 1st individual: {}\n".format(population[0]), 
      "2nd individual: {}\n".format(population[1]), 
      "3rd individual: {}\n".format(population[2]))

print("Average absolute error of initial population: {}".format(fitness_history[0]))

Create dataframe to store results

In [ ]:
comparison = pd.DataFrame()
comparison['target polynomial'] = ga.Y_TARGETS
comparison.name = 'Predicted vs. Target Polynomial'
comparison.index = ga.X_VALUES
comparison.index.name = 'x'
comparison.head()

Train GA for set configuration

In [ ]:
sweep = [0.015, 0.05, 0.1, 0.2]
results = {}

#for retain in sweep:
#for mutate in sweep:
#for random_select in [0.05, 0.1, 0.15, 0.2]:

lastGrade = 1

population = ga.createPopulation(count, individualMin, individualMax, individualLength)
fitness_history = [ga.getAvgPopulationFitness(population)]

while lastGrade != 0 and len(fitness_history) < maxGen:

    population = ga.evolve(population, 
                           retain=retain, 
                           mutate=mutate, 
                           select='roulette')

    lastGrade = ga.getAvgPopulationFitness(population)
    fitness_history.append(lastGrade)

#results['random_select' + str(random_select)] = fitness_history
#comparison['populationCount_' + str(count)] = ga.generatePredictions(population[0])
    
#comparison.head()

Plot error with generations

In [ ]:
aa = px.line(fitness_history)
aa.show()

## Plot polynomial generated by GA

In [ ]:
comparison['ga_polynomial'] = ga.generatePredictions(population[0])

In [ ]:
figPredictedPolynomial = px.line(comparison)
figPredictedPolynomial.show()

---

In [ ]:
# Calculate limit for number of parent chromosomes to select
poolSize = int(len(population) * retain)
logging.info("Desired mating pool size: {}".format(poolSize))

errors = np.array([])

# Get total fitness of population (cumulative sum of each individual's error)
for individual in population:
    # Get errors for each individual
    errors = np.append(errors, ga.getAvgAbsError(individual))

# Find sum of errors for all individuals
errorSum = np.sum(errors)

# Calculate selection probability for each individual as a proportion of total fitness for population: 1 - errors/errorSum
# Largest errors have smallest probabilities
selectProbabilities = np.divide(np.subtract(errorSum, errors), errorSum)

In [ ]:
errors

In [ ]:
selectProbabilities

In [ ]:
pairs = []
for err, prob in zip(errors, selectProbabilities):
    pairs.append((err, prob))
    
pairs = sorted(pairs)
pairs[:5]

In [ ]:
sum(selectProbabilities)

In [ ]:
sumErrors = sum(errors)
sumErrors

In [ ]:
abs(np.sum(selectProbabilities)/100)

In [ ]:
# Rank population based on individual grades
graded = [x[1] for x in pairs]

In [ ]:
selectProbabilitiesCS = np.cumsum(graded)
selectProbabilitiesCS = np.divide(selectProbabilitiesCS, 100)
selectProbabilitiesCS